In [ ]:
%matplotlib widget

In [ ]:
from fimpylab import LightsheetExperiment

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib import  pyplot as plt
import flammkuchen as fl
import pandas as pd

import pandas as pd

import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Libertinus Sans']

# %autoreload
from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, \
    fictive_trajectory_and_fit, qap_sorting_and_phase, fit_phase_neurons


In [ ]:
plt.close("all")
# path = Path("/Users/luigipetrucco/Desktop/source_data_batch1/210729_f6_cwccwck")
path = Path("/Users/luigipetrucco/Desktop/batch_210908/210908_f9_cwccw")
traces = fl.load(path / "filtered_traces.h5", "/detr")

reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values

df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()
exp = LightsheetExperiment(path)
fn = int(exp.fn)
beh_df = exp.behavior_log

In [ ]:
selected = fl.load(path / "selected.h5")

stim_start = traces.shape[0] // 2
t_lims = (100 * fn, stim_start)
t_slice = slice(*t_lims)

pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])

In [ ]:
# plt.close("all")
mot_t_slice = slice(stim_start, traces.shape[0])
f, axs = plt.subplots(1, 3, figsize=(7., 3.), sharex=True, sharey=True)
# plt.subplot(1,2,1)
# phase = np.angle((pcaed[:, 0]) + 1j * (pcaed[:, 1]))
axs[0].plot(pcaed[t_slice, 0], pcaed[t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs[0].scatter(pcaed[t_slice, 0], pcaed[t_slice, 1], 
                 c=phase[t_slice], lw=0.5, s=5, cmap="twilight",) 

axs[1].plot(pcaed[mot_t_slice, 0], pcaed[mot_t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs[1].scatter(pcaed[mot_t_slice, 0], pcaed[mot_t_slice, 1], 
                 c=phase[mot_t_slice], lw=0.5, s=5, cmap="twilight",) 

axs[2].plot(pcaed[:, 0], pcaed[:, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
axs[2].scatter(pcaed[:, 0], pcaed[:, 1], 
                 c=phase[:], lw=0.5, s=5, cmap="twilight",) 
sns.despine()

In [ ]:
phases_neuron, _ = fit_phase_neurons(traces[t_slice, selected], phase[t_slice])
perm = np.argsort(phases_neuron)

In [ ]:
from bouter.utilities import reliability

In [ ]:
def reshape_for_reliability(traces, n_reps):
    n_rep_pts = traces.shape[0] // n_reps
    return np.swapaxes(traces.reshape(n_reps, n_rep_pts, -1), 0, 1)
    

def reliability_from_traces(traces, n_reps):
    """Crop traces and returns reliability scores, assuming
    that the traces consist of consecutive blocks of a stimulus
    presented n_reps times.
    """
    resh = reshape_for_reliability(traces, n_reps)
    return reliability(resh)

In [ ]:
rels = reliability_from_traces(traces[stim_start:, :], 4)
reshaped = reshape_for_reliability(traces[stim_start:, :], 4)

stim_sel = np.argwhere(rels > 0.5)[:, 0]

In [ ]:
plt.close("all")
plt.figure(figsize=(4, 2))
plt.plot(reshaped[:, :, np.argsort(rels)[-1]])

In [ ]:
plt.figure(figsize=(4, 3))
plt.hist(rels, np.arange(0, 1, 0.04), lw=0, alpha=0.4, density=True, label="all ROIs")
plt.hist(rels[selected], np.arange(0, 1, 0.04), lw=0, alpha=0.4, density=True, label="ring")
plt.ylabel("Density (a.u.)")
plt.xlabel("Reliability")
plt.show()
sns.despine()
plt.legend()

In [ ]:
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")

In [ ]:
plt.figure(figsize=(4, 4))
plt.scatter(coords[:, 1], coords[:, 2], c=(0.9,)*3)
#plt.scatter(coords[stim_sel, 1], coords[stim_sel, 2],  
#            c=np.linspace(-np.pi, np.pi, perm_stim.max()+1)[np.argsort(perm_stim)] , cmap="twilight")
#plt.scatter(coords[stim_sel, 1], coords[stim_sel, 2],  
#            c=rels[stim_sel], cmap="viridis")
plt.scatter(coords[selected, 1], coords[selected, 2],  
            facecolors='none', edgecolors=cols[3], lw=0.5)
plt.scatter(coords[selected, 1], coords[selected, 2],  
            c=phases_neuron, cmap="twilight")
plt.axis("equal")
plt.axis("off")

In [ ]:
plt.figure(figsize=(4, 4))
plt.scatter(coords[:, 1], coords[:, 2], c=(0.9,)*3)
plt.scatter(coords[stim_sel, 1], coords[stim_sel, 2],  
            c=rels[stim_sel], cmap="viridis")
#plt.scatter(coords[stim_sel, 1], coords[stim_sel, 0]*10,  
#            c=rels[stim_sel], cmap="viridis")
plt.scatter(coords[selected, 1], coords[selected, 2],  
            facecolors='none', edgecolors=cols[3], lw=0.5)
#plt.scatter(coords[selected, 1], coords[selected, 2],  
#            c=np.linspace(-np.pi, np.pi, perm.max()+1)[np.argsort(perm)] , cmap="twilight")
plt.axis("equal")
plt.axis("off")

In [ ]:
perm_stim, com_phase = qap_sorting_and_phase(traces[:, stim_sel], t_lims=(5100, 10200))

plt.figure(figsize=(7, 3))
plt.subplot(121)
plt.imshow(np.corrcoef(traces[5100:, stim_sel].T)[perm_stim, :][:, perm_stim], 
           vmax=1, vmin=-1, cmap="RdBu_r")

plt.subplot(122)
plt.imshow(traces[:, stim_sel[perm_stim]].T, cmap="gray_r", aspect="auto", vmin=-2, vmax=3)